In [ ]:

import fitparse
from plotnine import *
from functions import *

import numpy as np
import pandas as pd
import scipy.io
from scipy import signal

import seaborn as sn
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import math


import os
os.chdir('C:/Users/Olsen/Desktop/Masteroppgave/Data/workfileexport - decompressed/')

fitfile = fitparse.FitFile('tp-3912799.2023-07-01-09-39-18-438Z.GarminPing.AAAAAGSf9EY7-gSP.FIT')


HRV_data = get_fit_hrv(fitfile)
HR_data = get_fit_hr(fitfile)


# Import data

In [ ]:
# To check RR from Polar (data from Project thesis for vertification of filterings)
def RR_data_txt(file_txt):
    # Extract RR and timestamps from txt file
    RRs_polar = []
    timestamps_polar = []

    f_file = open(file_txt,'r')
    next(f_file)
    # input format
    format = '%Y-%m-%dT%H:%M:%S.%f'

    for row in f_file:
        row = row.split(';')
        timestamps_polar.append(datetime.strptime(row[0], format))
        RRs_polar.append(int(row[1]))
    result = [timestamps_polar, RRs_polar]

    return result

# Filtering


In [ ]:
# artifact_correction_threshold = 0.05
# filtered_RRs = []

# for i in range(len(HRV_data)):
#   if HRV_data[(i-1)]*(1-artifact_correction_threshold) < HRV_data[i] < HRV_data[(i-1)]*(1+artifact_correction_threshold):
#         filtered_RRs.append(HRV_data[i])

# x = np.cumsum(filtered_RRs)


HRV_filtered = scipy.signal.medfilt(HRV_data, 3)
x = np.cumsum(HRV_filtered)


df = pd.DataFrame()
df['timestamp'] = x
df['RR'] = HRV_filtered

# ----------------------------------
#  Plotting
# ----------------------------------


fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df["timestamp"],
    y=df["RR"]*1000,
    name="HRV"
))

fig.update_layout(
    title="HRV",
    xaxis_title="Minuttes", yaxis_title="HRV [ms]",
    legend_title="Legend",
    font=dict(
        family="Courier New, monospace", size=18, color="RebeccaPurple"
    )
)

fig.show()

In [ ]:
def filtered(input_signal):
    filtered_signal = []
    for i in range(len(input_signal)):
        if input_signal[(i-1)]*0.5 < input_signal[i] or input_signal[i] < input_signal[(i-1)]*2:
            filtered_signal.append(input_signal[i])
    return  np.array(filtered_signal)

In [ ]:
def filtered_signal(input_signal):
    filtered_signal = []
    for i in range(len(input_signal)):
        if input_signal[(i-1)]*0.5 < input_signal[i] < input_signal[(i-1)]*1.25:
            filtered_signal.append(input_signal[i])
        else:
            filtered_signal.append(input_signal[i]/2)
            filtered_signal.append(input_signal[i]/2)
    return  filtered_signal

## Interpolering

In [ ]:
def interpolate_artifacts(data, threshold):
    interpolated_data = data.copy()  # Create a copy of the original data

    for i in range(1, len(data) - 1):
        if abs(data[i] - data[i - 1]) > threshold and abs(data[i] - data[i + 1]) > threshold:
            # If the difference between the current point and its neighbors exceeds the threshold
            interpolated_data[i] = (data[i - 1] + data[i + 1]) / 2  # Interpolate the value

    return interpolated_data


def interpolate_missing_packages(data):
    interpolated_data = data.copy()

    for i in range(2, len(data)):
        # Assuming the threshold to detect anomalies
        threshold = 1.25 * data[i - 1]

        if data[i] > threshold:
            # Interpolate missing values based on different scenarios of package loss
            if data[i] > 1.75 * data[i - 1]:
                interpolated_data[i] = (data[i - 1] + data[i - 2]) / 2
            elif data[i] > 2.75 * data[i - 1]:
                interpolated_data[i] = (data[i - 1] + data[i - 2] + data[i - 3]) / 3
        elif data[i] > threshold:
            interpolated_data[i] = 0
    return interpolated_data

# DFA computation methods

In [ ]:
def DFA(pp_values, lower_scale_limit, upper_scale_limit):
    scaleDensity = 30 # scales DFA is conducted between lower_scale_limit and upper_scale_limit
    m = 1 # order of polynomial fit (linear = 1, quadratic m = 2, cubic m = 3, etc...)

    # initialize, we use logarithmic scales
    start = np.log(lower_scale_limit) / np.log(10)
    stop = np.log(upper_scale_limit) / np.log(10)
    scales = np.floor(np.logspace(np.log10(math.pow(10, start)), np.log10(math.pow(10, stop)), scaleDensity))
    F = np.zeros(len(scales))
    count = 0

    for s in scales:
        rms = []
        # Step 1: Determine the "profile" (integrated signal with subtracted offset)
        x = pp_values
        y_n = np.cumsum(x - np.mean(x))
        # Step 2: Divide the profile into N non-overlapping segments of equal length s
        L = len(x)
        shape = [int(s), int(np.floor(L/s))]
        nwSize = int(shape[0]) * int(shape[1])
        # beginning to end, here we reshape so that we have a number of segments based on the scale used at this cycle
        Y_n1 = np.reshape(y_n[0:nwSize], shape, order="F")
        Y_n1 = Y_n1.T
        # end to beginning
        Y_n2 = np.reshape(y_n[len(y_n) - (nwSize):len(y_n)], shape, order="F")
        Y_n2 = Y_n2.T
        # concatenate
        Y_n = np.vstack((Y_n1, Y_n2))

        # Step 3: Calculate the local trend for each 2Ns segments by a least squares fit of the series
        for cut in np.arange(0, 2 * shape[1]):
            xcut = np.arange(0, shape[0])
            pl = np.polyfit(xcut, Y_n[cut,:], m)
            Yfit = np.polyval(pl, xcut)
            arr = Yfit - Y_n[cut,:]
            rms.append(np.sqrt(np.mean(arr * arr)))

        if (len(rms) > 0):
            F[count] = np.power((1 / (shape[1] * 2)) * np.sum(np.power(rms, 2)), 1/2)
        count = count + 1

    pl2 = np.polyfit(np.log2(scales), np.log2(F), 1)
    alpha = pl2[0]
    return alpha





def computeFeatures(df):
  features = []
  step = 120
  for index in range(0, int(round(np.max(x)/step))):
      
      array_rr = df.loc[(df['timestamp'] >= (index*step)) & (df['timestamp'] <= (index+1)*step), 'RR']*1000
      # compute heart rate
      heartrate = round(60000/np.mean(array_rr), 2)
      # compute rmssd
      NNdiff = np.abs(np.diff(array_rr))
      rmssd = round(np.sqrt(np.sum((NNdiff * NNdiff) / len(NNdiff))), 2)
      # compute sdnn 
      sdnn = round(np.std(array_rr), 2)
      #dfa, alpha 1
      alpha1 = DFA(array_rr.to_list(), 4, 16)

      curr_features = {
          'timestamp': index*step,
          'heartrate': heartrate,
          'rmssd': rmssd,
          'sdnn': sdnn,
          'alpha1': alpha1,
      }

      features.append(curr_features)

  features_df = pd.DataFrame(features)
  return features_df    


features_df = computeFeatures(df)
features_df.head()

fig, ax1 = plt.subplots(figsize=(10, 5))

# Plot the synthetic signal on the primary y-axis
ax1.set_xlabel('Time')
ax1.set_ylabel('Signal', color='tab:blue')
ax1.plot(df["timestamp"], 60/df["RR"], color='tab:blue')
ax1.tick_params(axis='y', labelcolor='tab:blue')

# Plot DFA Alpha 1 on the secondary y-axis
ax2 = ax1.twinx()
ax2.set_ylabel('DFA Alpha 1', color='tab:red')
ax2.plot(features_df["timestamp"], features_df["alpha1"], "-o", color='tab:red', label='DFA Alpha 1')
ax2.tick_params(axis='y', labelcolor='tab:red')
ax2.fill_between(x, 0.75, 0.5, color='C0', alpha=0.2)
ax2.fill_between(x, 0.5, 0, color='C0', alpha=0.1)

plt.title('HR and DFA Alpha 1 over Time')
plt.grid(True)
plt.show()

print(np.mean(features_df['alpha1']))

In [ ]:
# def DFA(pp_values, lower_scale_limit, upper_scale_limit):
#     scaleDensity = 30 # scales DFA is conducted between lower_scale_limit and upper_scale_limit
#     m = 1 # order of polynomial fit (linear = 1, quadratic m = 2, cubic m = 3, etc...)

#     initialize, we use logarithmic scales
#     start = np.log(lower_scale_limit) / np.log(10)
#     stop = np.log(upper_scale_limit) / np.log(10)
#     scales = np.floor(np.logspace(np.log10(math.pow(10, start)), np.log10(math.pow(10, stop)), scaleDensity))
#     F = np.zeros(len(scales))
#     count = 0

#     for s in scales:
#         rms = []
#         Step 1: Determine the "profile" (integrated signal with subtracted offset)
#         x = pp_values
#         y_n = np.cumsum(x - np.mean(x))
#         Step 2: Divide the profile into N non-overlapping segments of equal length s
#         L = len(x)
#         shape = [int(s), int(np.floor(L/s))]
#         nwSize = int(shape[0]) * int(shape[1])
#         beginning to end, here we reshape so that we have a number of segments based on the scale used at this cycle
#         Y_n1 = np.reshape(y_n[0:nwSize], shape, order="F")
#         Y_n1 = Y_n1.T
#         end to beginning
#         Y_n2 = np.reshape(y_n[len(y_n) - (nwSize):len(y_n)], shape, order="F")
#         Y_n2 = Y_n2.T
#         concatenate
#         Y_n = np.vstack((Y_n1, Y_n2))

#         Step 3: Calculate the local trend for each 2Ns segments by a least squares fit of the series
#         for cut in np.arange(0, 2 * shape[1]):
#             xcut = np.arange(0, shape[0])
#             pl = np.polyfit(xcut, Y_n[cut,:], m)
#             Yfit = np.polyval(pl, xcut)
#             arr = Yfit - Y_n[cut,:]
#             rms.append(np.sqrt(np.mean(arr * arr)))

#         if (len(rms) > 0):
#             F[count] = np.power((1 / (shape[1] * 2)) * np.sum(np.power(rms, 2)), 1/2)
#         count = count + 1

#     pl2 = np.polyfit(np.log2(scales), np.log2(F), 1)
#     alpha = pl2[0]
#     return alpha


# def computeFeatures(df):
#   features = []
#   step = 120
#   x = np.multiply(df["RR"][0:step], 1000)

#   for index in range(0,int(round(np.max(x)/step))):
#       array_rr = df.loc[(df['timestamp'] >= (index*step)) & (df['timestamp'] <= (index+1)*step), 'RR'*1000]
#       compute heart rate
#       heartrate = round(60000/np.mean(array_rr), 2)
#       compute rmssd
#       NNdiff = np.abs(np.diff(array_rr))
#       rmssd = round(np.sqrt(np.sum((NNdiff * NNdiff) / len(NNdiff))), 2)
#       compute sdnn 
#       sdnn = round(np.std(array_rr), 2)
#       dfa, alpha 1
#       alpha1 = DFA(array_rr.to_list(), 4, 16)

#       curr_features = {
#           'timestamp': index*step,
#           'heartrate': heartrate,
#           'rmssd': rmssd,
#           'sdnn': sdnn,
#           'alpha1': alpha1,
#       }

#       features.append(curr_features)

#   features_df = pd.DataFrame(features)
#   return features_df 

In [ ]:
def dfa_continous(data, window_size):
    n = len(data)
    scales = np.logspace(1, np.log10(window_size), num=20, dtype=int)
    fluctuations = []

    for i in range(0, n - window_size + 1):
        current_window = data[i:i + window_size]
        window_n = len(current_window)

        scale_fluctuations = []

        for scale in scales:
            num_segments = window_n // scale
            fluctuation = 0.0

            for segment in range(num_segments):
                start = segment * scale
                end = (segment + 1) * scale
                segment_data = current_window[start:end]
                segment_fit = np.polyfit(np.arange(start, end), segment_data, 1)
                segment_trend = np.polyval(segment_fit, np.arange(start, end))
                segment_fluctuation = np.sqrt(np.mean((segment_data - segment_trend) ** 2))
                fluctuation += segment_fluctuation

            fluctuation /= num_segments
            scale_fluctuations.append(fluctuation)

        alpha, _ = np.polyfit(np.log10(scales), np.log10(scale_fluctuations), 1)
        fluctuations.append(abs(alpha))
    return fluctuations
